# Geospatial Visualization Demo

This notebook demonstrates mapping and spatial analysis:
- Interactive point maps with Folium
- Heatmaps
- Marker clustering
- Choropleth maps
- Static maps with GeoPandas

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent.parent))

from scripts.visualization import (
    point_map, heatmap, cluster_map, choropleth_map, static_map,
    set_theme, get_map_theme
)
from scripts.data_storage import save_geoparquet, load_geoparquet
from scripts.normalization import standardize_coordinates

# Set visualization theme
set_theme('light')

# Note: Interactive maps require an internet connection for tile loading.
# For fully offline work, use static_map() which renders with matplotlib.
print("Map tile sources used by default:")
print("  - OpenStreetMap: requires internet")
print("  - CartoDB: requires internet")
print("  - Use static_map() for fully offline maps")

In [ ]:
# Generate sample monitoring site data for the Colorado River Basin
np.random.seed(42)
n_sites = 100

# Colorado River Basin approximate bounds
# Upper: roughly 37-42°N, 105-112°W
# Lower: roughly 31-37°N, 110-117°W

sites = pd.DataFrame({
    'site_id': [f'CRB_{i:04d}' for i in range(n_sites)],
    'latitude': np.concatenate([
        np.random.uniform(37, 41, n_sites//2),  # Upper basin
        np.random.uniform(32, 37, n_sites//2),  # Lower basin
    ]),
    'longitude': np.concatenate([
        np.random.uniform(-112, -106, n_sites//2),  # Upper basin
        np.random.uniform(-116, -110, n_sites//2),  # Lower basin
    ]),
    'basin': ['Upper Colorado'] * (n_sites//2) + ['Lower Colorado'] * (n_sites//2),
    'site_type': np.random.choice(['Groundwater Well', 'Stream Gage', 'Reservoir'], n_sites),
    'depth_ft': np.random.exponential(100, n_sites),
    'water_quality_index': np.random.uniform(50, 100, n_sites),
})

sites.head()

## Interactive Point Maps

In [ ]:
# Basic point map
m = point_map(
    sites,
    lat_col='latitude',
    lon_col='longitude',
    popup_cols=['site_id', 'basin', 'site_type', 'water_quality_index'],
    center=[36.5, -111],
    zoom=6,
)
m

In [ ]:
# Colored by basin
m = point_map(
    sites,
    lat_col='latitude',
    lon_col='longitude',
    color_col='basin',
    popup_cols=['site_id', 'basin', 'site_type'],
    center=[36.5, -111],
    zoom=6,
)
m

In [ ]:
# Sized by water quality index, colored by site type
m = point_map(
    sites,
    lat_col='latitude',
    lon_col='longitude',
    color_col='site_type',
    size_col='water_quality_index',
    popup_cols=['site_id', 'site_type', 'water_quality_index'],
    center=[36.5, -111],
    zoom=6,
    tiles='CartoDB positron',
)
m

## Heatmaps

In [ ]:
# Heatmap weighted by water quality index
m = heatmap(
    sites,
    lat_col='latitude',
    lon_col='longitude',
    value_col='water_quality_index',
    center=[36.5, -111],
    zoom=6,
    radius=20,
    tiles='CartoDB dark_matter',
)
m

In [ ]:
# Density heatmap (no weights)
m = heatmap(
    sites,
    lat_col='latitude',
    lon_col='longitude',
    center=[36.5, -111],
    zoom=6,
    radius=15,
    blur=15,
)
m

## Marker Clustering

In [ ]:
# Clustered markers for many points
m = cluster_map(
    sites,
    lat_col='latitude',
    lon_col='longitude',
    popup_cols=['site_id', 'basin', 'site_type', 'water_quality_index'],
    center=[36.5, -111],
    zoom=6,
)
m

## Working with GeoDataFrames

In [ ]:
# Convert to GeoDataFrame
gdf = standardize_coordinates(sites, create_geometry=True)
print(f"GeoDataFrame with CRS: {gdf.crs}")
gdf.head()

In [ ]:
# Save as GeoParquet
output_path = save_geoparquet(gdf, 'colorado_basin_sites')
print(f"Saved to: {output_path}")

In [ ]:
# Load GeoParquet and verify geometry preserved
loaded_gdf = load_geoparquet('colorado_basin_sites')
print(f"Loaded GeoDataFrame with CRS: {loaded_gdf.crs}")
print(f"Geometry column: {loaded_gdf.geometry.name}")

## Static Maps with GeoPandas

In [ ]:
# Basic static map
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 8))
gdf.plot(
    ax=ax,
    column='water_quality_index',
    cmap='RdYlGn',
    legend=True,
    legend_kwds={'label': 'Water Quality Index'},
    markersize=50,
)
ax.set_title('Monitoring Sites by Water Quality Index')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.tight_layout()

In [ ]:
# Load US states for context (from Natural Earth via geopandas)
try:
    # This uses the Natural Earth dataset bundled with geopandas
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    usa = world[world['name'] == 'United States of America']
    
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Plot USA boundary
    usa.boundary.plot(ax=ax, color='gray', linewidth=0.5)
    
    # Plot sites
    gdf.plot(
        ax=ax,
        column='basin',
        cmap='Set1',
        legend=True,
        markersize=30,
        alpha=0.7,
    )
    
    # Zoom to Colorado Basin area
    ax.set_xlim(-118, -104)
    ax.set_ylim(30, 43)
    
    ax.set_title('Colorado River Basin Monitoring Sites')
    plt.tight_layout()
    
except Exception as e:
    print(f"Could not load Natural Earth data: {e}")
    print("This is optional - your point data still works fine")

## Spatial Analysis Examples

In [ ]:
# Buffer around sites (e.g., 50km influence zone)
# First reproject to a projected CRS for accurate distance calculations
gdf_projected = gdf.to_crs(epsg=32612)  # UTM Zone 12N (covers Colorado Basin)

# Create 50km buffers
gdf_projected['buffer_50km'] = gdf_projected.geometry.buffer(50000)  # 50km in meters

fig, ax = plt.subplots(figsize=(10, 8))
gdf_projected.set_geometry('buffer_50km').plot(ax=ax, alpha=0.3, color='blue')
gdf_projected.plot(ax=ax, color='red', markersize=10)
ax.set_title('50km Buffer Zones Around Monitoring Sites')
plt.tight_layout()

In [ ]:
# Calculate statistics by basin
basin_stats = gdf.groupby('basin').agg({
    'site_id': 'count',
    'water_quality_index': ['mean', 'std', 'min', 'max'],
    'depth_ft': 'mean',
}).round(2)

basin_stats.columns = ['_'.join(col).strip('_') for col in basin_stats.columns]
basin_stats

## Saving Maps

In [ ]:
# Save interactive map to HTML
output_dir = Path.cwd().parent.parent / 'data' / 'outputs'

m = point_map(
    sites,
    color_col='basin',
    size_col='water_quality_index',
    popup_cols=['site_id', 'basin', 'site_type', 'water_quality_index'],
    center=[36.5, -111],
    zoom=6,
    save_path=output_dir / 'basin_sites_map.html'
)

print(f"Interactive map saved to: {output_dir / 'basin_sites_map.html'}")

In [ ]:
# Save static map as PNG
fig, ax = plt.subplots(figsize=(12, 10))
gdf.plot(
    ax=ax,
    column='water_quality_index',
    cmap='RdYlGn',
    legend=True,
    markersize=50,
)
ax.set_title('Water Quality Index by Site')
fig.savefig(output_dir / 'wqi_static_map.png', dpi=150, bbox_inches='tight')
print(f"Static map saved to: {output_dir / 'wqi_static_map.png'}")
plt.close()